# Decorator Examples

- `@wraps` decorator from `functools` module is used to preserve the metadata of the original function
    - https://docs.python.org/3/library/functools.html#functools.wraps
    - https://stackoverflow.com/questions/308999/what-does-functools-wraps-do
- decorators can be stacked 
    - care for order of execution
- Decorators with arguments
    - https://realpython.com/primer-on-python-decorators/#decorators-with-arguments
- Default Decorators
    - from decorators import debug, timer


In [134]:
import time
from functools import wraps

## Debug Decorator

In [135]:
def debug(fn):
    """
    Debugger decorator
    """

    # @wraps is itself a decorator which copies over the original parameters of funtion fn
    #   - e.g. name, docstring
    @wraps(fn)
    def debugger(*args, **kwargs):
        """
        Closure of decorator, running args of passed function
        """
        print(f"Args: {args}")
        print(f"Kwargs: {kwargs}")
        print(f"Function '{fn.__name__}' called.")
        fn_result = fn(*args, **kwargs)
        print(f"Function {fn.__name__} returns {fn_result}")
        return fn_result
    return debugger

In [136]:
@debug
def do_something1(a, b, c=None):
    """
    Dummy function 1
    """
    return a+b if c else 0


@debug
def do_something(a, b, c=None):
    """
    Dummy function 2
    """
    return a-b if c else 0

In [137]:
do_something1(12, 5, c=1)

# test what happen if  @wrapper is removed!!
print(do_something1.__name__)
print(do_something1.__doc__)

Args: (12, 5)
Kwargs: {'c': 1}
Function 'do_something1' called.
Function do_something1 returns 17
do_something1

    Dummy function 1
    


## Timer Decorator

In [138]:
def timing(fn):
    @wraps(fn)
    def timer(*args, **kwargs):
        print("Start timer")
        start = time.perf_counter()
        fn_result = fn(*args, **kwargs)
        end = time.perf_counter()
        duration = end-start
        print(f"Execution of function '{fn.__name__}' took: {duration} s")
        return fn_result
    return timer

In [139]:
@timing  # timing(debug)
@debug   # debug(iterate_stuff)
def iterate_stuff(n):
    result = 0
    for i in range(n):
        result += i
    return result

In [140]:
iterate_stuff(9999999)

Start timer
Args: (9999999,)
Kwargs: {}
Function 'iterate_stuff' called.
Function iterate_stuff returns 49999985000001
Execution of function 'iterate_stuff' took: 1.417945499997586 s


49999985000001

## Decorator Factory

- adding arguments to decorator
    - https://realpython.com/primer-on-python-decorators/#decorators-with-arguments

### Example - repeat

In [141]:
def repeat_twice(fn):

    @wraps(fn)
    def repeat(*args, **kwargs):
        fn(*args, **kwargs)
        fn(*args, **kwargs)
    return repeat

In [142]:
@repeat_twice
def hello_world(name):
    print(f"hello {name}")

In [143]:
hello_world("Olli")

hello Olli
hello Olli


In [144]:
def repeat(val):

    def repeat_twice(fn):
        @wraps(fn)
        def repeat(*args, **kwargs):
            for _ in range(val):
                fn(*args, **kwargs)
        return repeat
    return repeat_twice

In [145]:
@repeat(4)
def hello_world(name):
    print(f"hello {name}")

In [146]:
hello_world("Olli")

hello Olli
hello Olli
hello Olli
hello Olli


### Example - timing

In [147]:
def timing_extended(use_ns=False):
    if use_ns == True:
        time_fn = time.perf_counter_ns
        scale = "ns"
    else:
        time_fn = time.perf_counter
        scale = "s"

    def timing(fn):
        @wraps(fn)
        def timer(*args, **kwargs):
            print("Start timer")
            start = time_fn()
            fn_result = fn(*args, **kwargs)
            end = time_fn()
            duration = end-start
            print(
                f"Execution of function '{fn.__name__}' took: {duration} {scale}")
            return fn_result
        return timer
    return timing

In [148]:
@timing_extended(use_ns=True)
def iter(val):
    res = 0
    for i in range(val):
        res += i
    return res

In [149]:
iter(99999)

Start timer
Execution of function 'iter' took: 14502800 ns


4999850001